In [2]:
import pandas as pd

from konlpy.tag import Okt
from konlpy.tag import Hannanum
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [606]:
hannanum = Hannanum()

In [36]:
print(hannanum.analyze(u'롯데마트의 흑마늘 양념 치킨이 논란이 되고 있다.'))

[[[('롯데마트', 'ncn'), ('의', 'jcm')], [('롯데마트의', 'ncn')], [('롯데마트', 'nqq'), ('의', 'jcm')], [('롯데마트의', 'nqq')]], [[('흑마늘', 'ncn')], [('흑마늘', 'nqq')]], [[('양념', 'ncn')]], [[('치킨', 'ncn'), ('이', 'jcc')], [('치킨', 'ncn'), ('이', 'jcs')], [('치킨', 'ncn'), ('이', 'ncn')]], [[('논란', 'ncpa'), ('이', 'jcc')], [('논란', 'ncpa'), ('이', 'jcs')], [('논란', 'ncpa'), ('이', 'ncn')]], [[('되', 'nbu'), ('고', 'jcj')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecc')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecs')], [('되', 'nbu'), ('이', 'jp'), ('고', 'ecx')], [('되', 'paa'), ('고', 'ecc')], [('되', 'paa'), ('고', 'ecs')], [('되', 'paa'), ('고', 'ecx')], [('되', 'pvg'), ('고', 'ecc')], [('되', 'pvg'), ('고', 'ecs')], [('되', 'pvg'), ('고', 'ecx')], [('되', 'px'), ('고', 'ecc')], [('되', 'px'), ('고', 'ecs')], [('되', 'px'), ('고', 'ecx')]], [[('있', 'paa'), ('다', 'ef')], [('있', 'px'), ('다', 'ef')]], [[('.', 'sf')], [('.', 'sy')]]]


In [607]:
okt = Okt()

In [608]:
# 데이터 가져오기
hotel_info_df = pd.read_csv('../../dataset/final/final_hotel_info.csv', encoding='utf-8')
hotel_review_df = pd.read_csv('../../dataset/final/final_hotel_review.csv', encoding='utf-8')

In [4]:
hotel_info_df.head(3)

,region,hotel_id,hote_name,hotel_link,hotel_img_link
0,서울,1,콘래드 서울,https://www.booking.com/hotel/kr/conrad-seoul....,https://t-cf.bstatic.com/xdata/images/hotel/sq...
1,서울,2,시그니엘,https://www.booking.com/hotel/kr/signiel-seoul...,https://t-cf.bstatic.com/xdata/images/hotel/sq...
2,서울,3,그랜드 하얏트 서울,https://www.booking.com/hotel/kr/grand-hyatt-s...,https://t-cf.bstatic.com/xdata/images/hotel/sq...


In [5]:
hotel_review_df.head(3)

,review_id,date,review,hotel_index,initial_label
0,1,2021-06,조식이 세미 뷔페식이었어요. \n여자친구는 건강식 전 펜케익을 먹었었는데 맛은 매우...,1,1
1,2,2022-01,우선 예약 상 문제가 있었던 부분에 대해 콘래드 측에서 즉각적인 대응을 해주셔서\n...,1,1
2,3,2022-02,쇼핑몰과 연결되어 있어 이동이 편했습니다.,1,1


In [609]:
hotel_review_df.rename(columns = {'hotel_index' : 'hotel_id'}, inplace=True)

In [3]:
def cleansing(document, pos):
    okt = Okt()

    # document normalization
    norm_doc = okt.normalize(document)
    token_list = okt.pos(norm_doc, join=True)
    targets = pos
    #targets = ["Adjective", "Adverb", "Noun", "Verb"]

    # targets에 해당되는 token만 추출해 저장
    doc_tokenized = []
    for token in token_list:
        for target in targets:
            if target in token:
                dash_index = token.index("/")
                doc_tokenized.append(token[0:dash_index])

    # stopwords 제거한 token 저장
    stop_words = "이	있 하 것 들 그 되 수 이 보 않 없 나 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 경우 명 이런 앞 보이 번 나 다른 어떻 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓"
    stop_words = set(stop_words.split(" "))
    cleansed_doc = [word for word in doc_tokenized if not word in stop_words]

    return cleansed_doc

In [612]:
# review_list를 가져와서 형태소 추출 후 tag(region, hotel_id)와 함께 tagged_corpus_list에 저장
def tag_corpus(hotel_info_df, hotel_review_df, pos):
    positive_hotel_review_df = hotel_review_df[hotel_review_df['label'] == 1]
    reindex_hotel_info_df = hotel_info_df.set_index('hotel_id')
    tagged_corpus_list = []

    print(f'start tagging pid : {os.getpid()}')
    for df in positive_hotel_review_df.itertuples():
        #index = df.Index
        review_id = df.review_id
        hotel_id = df.hotel_id
        review = df.contents

        try:
            region = reindex_hotel_info_df.loc[hotel_id]['region']
            cleansed_doc = cleansing(review, pos)
            tagged_line = TaggedDocument(
                tags=[f'{region}@{hotel_id}@{review_id}'], words=cleansed_doc)
            tagged_corpus_list.append(tagged_line)
        except Exception as e:
            print('Error in tagged_corpus', e)
            continue
        #print(f'---{round((index / len(hotel_review_df)) * 100 ,3)}%---')

    return tagged_corpus_list

In [613]:
def build_model(model_name, tagged_corpus_list, vector_size, window, min_count, epochs):
    # 모델 생성
    d2v_model = Doc2Vec(vector_size=vector_size, window=window, workers=8,
                        min_count=len(tagged_corpus_list) // min_count)
    # 단어 빌드
    print(f"start model build pid : {os.getpid()}")
    d2v_model.build_vocab(tagged_corpus_list)
    # 학습
    print(f"start training pid : {os.getpid()}")
    d2v_model.train(tagged_corpus_list,
                    total_examples=d2v_model.corpus_count, epochs=epochs)
    # 저장
    #d2v_model.save(f'app/service/ai/{model_name}.model')
    d2v_model.save(f'./AI/models/{model_name}.model')

    return model_name

In [614]:
def save_model(hotel_info_df, hotel_review_df, model_name, pos, token_min, token_max, vector_size, window, min_count, epochs):
    try:
        tagged_corpus_list = tag_corpus(hotel_info_df, hotel_review_df, pos)
        # 너무 길거나 짧은 리뷰 제거
        filtered_tagged_corpus_list = [x for x in tagged_corpus_list if (
            len(x[0]) >= token_min) and (len(x[0]) <= token_max)]

        model_name = build_model(
            model_name, filtered_tagged_corpus_list, vector_size, window, min_count, epochs)
    except Exception as e:
        print(e)
        return(f'{model_name} save fail')

    return f"{model_name} save success"

In [13]:
filtered_tagged_corpus_list = [x for x in tagged_corpus_list if (len(x[0]) >= 5) and (len(x[0]) <= 100)]

In [14]:
filtered_tagged_corpus_list[:3]

[TaggedDocument(words=['조식', '세미', '뷔페', '이었어요', '여자친구', '건강', '펜', '케익', '먹었었는데', '맛', '매우', '만족스러웠어요', '고급스러운', '호텔', '조식', '원하신다면', '추천', '합니다'], tags=['1@1']),
 TaggedDocument(words=['우선', '예약', '상', '있었던', '부분', '대해', '콘래드', '측', '즉각', '대응', '해주셔서', '만족스러운', '서비스', '받은', '대해', '감사', '드립니다', '몰이', '지하', '바로', '연결', '되고', '이를', '통해', '현대', '지하', '이용', '할', '있어', '편안한', '쇼핑', '가능하며', '한강', '도보', '갈', '있을', '의', '거리', '입니다', '한강', '망', '가능한', '방', '이를', '보고', '있는', '편안한', '휴식', '되며', '실내', '수영장', '레인', '구분', '통해', '가족', '이용', '또는', '자유', '수영', '하기', '좋은', '환경', '이었습니다', '조식', '신선한', '재료', '사용', '했다는', '느낌', '받아', '만족', '이용', '했습니다'], tags=['1@2']),
 TaggedDocument(words=['쇼핑몰', '연결', '되어', '있어', '이동', '편했습니다'], tags=['1@3'])]

In [15]:
# 모델 생성
d2v_model = Doc2Vec(vector_size=300, window=3, workers=8, min_count = len(tagged_corpus_list) // 1000)

In [16]:
# 단어 빌드
d2v_model.build_vocab(tagged_corpus_list)

In [17]:
# 학습
d2v_model.train(tagged_corpus_list, total_examples=d2v_model.corpus_count, epochs=30)

In [18]:
d2v_model.save('../model/d2v.model')

In [8]:
d2v_model = Doc2Vec.load('..//models/positive_d2v.model')

In [18]:
print(len(filtered_tagged_corpus_list))
print(len(tagged_corpus_list))

79117
80851


In [6]:
# 테스트
user_input = '바다뷰가 좋고 직원이 친절했어요'
pos = ["Adjective", "Noun", "Verb"]
cleansed_test_target = cleansing(user_input, pos)
noun_user_input = cleansed_test_target
print(noun_user_input)

['바다', '뷰', '좋고', '직원', '친절했어요']


In [10]:
# 유저 입력값의 벡터 생성
user_input_vector = d2v_model.infer_vector(noun_user_input)

In [11]:
# 유사도가 높은 리뷰 확인
most_similar_docs = d2v_model.docvecs.most_similar([user_input_vector], topn=len(d2v_model.docvecs))

In [12]:
most_similar_docs

[('부산@235@30939', 0.7315610647201538),
 ('부산@299@37964', 0.7225601673126221),
 ('서울@37@4088', 0.7216836214065552),
 ('제주@134@15911', 0.7201308012008667),
 ('제주@121@13894', 0.7124667167663574),
 ('제주@141@17119', 0.7059019804000854),
 ('강원@360@59189', 0.6965150833129883),
 ('강원@346@56562', 0.6878955364227295),
 ('강원@360@59357', 0.6875061392784119),
 ('제주@208@21369', 0.687432587146759),
 ('제주@139@16777', 0.6826157569885254),
 ('제주@107@11647', 0.6819884777069092),
 ('부산@290@39963', 0.6789589524269104),
 ('서울@49@7063', 0.677105188369751),
 ('부산@238@31846', 0.6751830577850342),
 ('강원@360@59098', 0.674651026725769),
 ('여수@428@74374', 0.6739303469657898),
 ('강원@360@58901', 0.6710131168365479),
 ('서울@23@2439', 0.6639823913574219),
 ('강원@360@58566', 0.6586458683013916),
 ('제주@197@20742', 0.6559953093528748),
 ('부산@250@33960', 0.6540780663490295),
 ('부산@237@31421', 0.6483864784240723),
 ('강원@380@61822', 0.6480354070663452),
 ('부산@251@34253', 0.6467089653015137),
 ('강원@342@54424', 0.64577329158782

In [23]:
# 몇 개의 호텔을 추천할지
number_top = 100

# 몇 개의 리뷰를 기준으로 할지
number_review = 4

# list의 길이를 기준으로 정렬하기 위해 Series 사용
similar_review_hotel = pd.Series([], dtype='object')

count_pass_hotel = 0
for review, similarity in most_similar_docs:
    hotel, index = review.split("@")

    if hotel not in similar_review_hotel:
        similar_review_hotel[hotel] = [index]
    else:
        similar_review_hotel[hotel].append(index)
    
    if len(similar_review_hotel[hotel]) == number_review:
        count_pass_hotel += 1
    if count_pass_hotel >= number_top:
        break

In [24]:
similar_review_hotel

62                                    [5436, 5463, 9966]
309    [32106, 32223, 32192, 31729, 43326, 32182, 323...
441    [59666, 59676, 59029, 59318, 59244, 59222, 697...
383           [41180, 41152, 52140, 52099, 41162, 52135]
539                                [77067, 80304, 77068]
                             ...                        
349                                       [36973, 36947]
338                                              [47970]
145                                              [23446]
233                                              [20098]
35                                                [2832]
Length: 287, dtype: object

In [25]:
recomended_hotel = similar_review_hotel.sort_values(key=lambda x: x.str.len(), ascending=False).head(number_top)

In [26]:
recomended_hotel

441    [59666, 59676, 59029, 59318, 59244, 59222, 697...
429    [57260, 57740, 57299, 57683, 68059, 57813, 577...
423    [55269, 55426, 55563, 55500, 55447, 65618, 554...
324    [35122, 46191, 34932, 46241, 46667, 34880, 463...
422    [54937, 54423, 55011, 54698, 54625, 64858, 548...
                             ...                        
405                         [53494, 42668, 42722, 53464]
253                         [21009, 20991, 29989, 29985]
546                         [77498, 80736, 77520, 80753]
215                         [18944, 18957, 28380, 28381]
421                         [64304, 54377, 54277, 64257]
Length: 100, dtype: object

In [27]:
# 추천된 호텔과 리뷰 확인
for hotel_id, review_id_list in list(recomended_hotel.items()):
    print(f'----- {hotel_info_df.loc[int(hotel_id),"hote_name"]} -----')
    for review_id in review_id_list[:number_review]:
        print(hotel_review_df.loc[int(review_id)]['review'])
    print('')

----- 씨크루즈호텔 속초 -----
바다뷰와 바다와 가까워서 산책하기 좋아요~~(바다쪽 방은 일출을 방에서 볼수 있어서 좋아요)
바다도 잘 보이고 깨끗하고 좋았습니다. 주차장 시설도 잘 되어있고 전체적으로 좋았어요.
위치가 좋음.  방이 깨끗하고 침대가 큼 해돋이 전망이 최고임
위치는 좋았으나 주변 상가환경이 다소 산만한 것이 아쉬움

----- 더화이트 호텔 -----
부킹닷컴 통해 할인 받았습니다 시설은 깨끗하고 좋습니다  천장이 높아서 넓게 느껴져 정말 좋았습니다
다락처럼 복층으로 되어있어 애들이 좋아할것 같아요 ~ (밑에층 이나 옆방들은 소음에 괴롭겠지만 ㅠ) 테라스에서 바라보는 뷰와 공기가 참 좋아요~ 위치도 괜찮고 가성비면에서는 나쁘지 않아요
가성비가 좋고 위치가 외진만큼 호텔내 부대시설이 잘되어있음. 씨유 포함 음식점 다수. 전체적인 위생상태가 깨끗하고 호텔인 만큼 호텔다운 구성을 갖춤 (어메니티 투숙객 서비스 등) 층고가 높아 객실이 작아도 답답하지 않고 침구류도 좋았음. 산속에 있어서 한여름에도 매우 시원하고 6월 말임에도 모기등 벌레가 별로 없었음. 양떼목장 등 관광지와도 가까움
층고가 높아서 확ㅡ트인 느낌과 깔끔한 분위기는 첫인상을 매우 좋게 만들어줍니다^^ 복층구조인데 ㅣㅣ살 조카와 함께했지만 굳이 올라갈 일은 없었네요. 근데 그런 거 있잖아요? 다락방있는 것만으로도 어른들에겐 옛운치 아이들에겐 나만의 공간ㅡ처음엔 거기가 자신의 아지트라고 하더니 허리를 굽혀 다녀야하는 2층이 불편했는지 더이상 오르지 않음^^ ㅡ고객의 감성을 신경쓴 느낌이라 아주 만족해했습니다. 아~그리고 조용해서 아주 좋았습니다~방배정 잘못 받으면 돈 내고 스트레스 쌓아서 가는 불운ㅜㅜ 아주 조용하고 시원하고 깔끔하고 함께 한 할머니와 조카군 모두 다음에 또 와보자 했을 정도입니다. 가격대비 최고라 감히 적어봅니다.

----- 롯데 리조트 속초 -----
해변과 가까이 있어서 산책하기가 매우 좋았음. 호텔 주위에 관광지가 가깝게 위치해 있어서 쉽게 관광지를 둘러 보

In [36]:
return_data = []
for hotel_id, review_id_list in recomended_hotel.items():
    hotel_dict = {}
    hotel_dict["hotel_id"] = hotel_id
    hotel_dict['review_id'] = []
 
    for review_id in review_id_list[:len(recomended_hotel[-1])]:
        #print(review_id)
        hotel_dict['review_id'].append(review_id)
    
    return_data.append(hotel_dict)

In [ ]:

# 추천된 호텔과 리뷰 확인
for hotel_id, review_id_list in return_data:
    print(f'----- {hotel_info_df.loc[int(hotel_id),"hote_name"]} -----')
    for review_id in review_id_list[:number_review]:
        print(hotel_review_df.loc[int(review_id)]['review'])
    print('')

In [63]:
return_data

[{'hotel_id': '441', 'review_id': ['59666', '59676', '59029', '59318']},
 {'hotel_id': '429', 'review_id': ['57260', '57740', '57299', '57683']},
 {'hotel_id': '423', 'review_id': ['55269', '55426', '55563', '55500']},
 {'hotel_id': '324', 'review_id': ['35122', '46191', '34932', '46241']},
 {'hotel_id': '422', 'review_id': ['54937', '54423', '55011', '54698']},
 {'hotel_id': '309', 'review_id': ['32106', '32223', '32192', '31729']},
 {'hotel_id': '518', 'review_id': ['75062', '78487', '74965', '78656']},
 {'hotel_id': '366', 'review_id': ['39222', '39419', '50404', '39377']},
 {'hotel_id': '455', 'review_id': ['60969', '71952', '71670', '71627']},
 {'hotel_id': '322', 'review_id': ['34314', '34421', '34466', '34420']},
 {'hotel_id': '442', 'review_id': ['59926', '59792', '59958', '70590']},
 {'hotel_id': '451', 'review_id': ['71163', '60610', '71518', '71151']},
 {'hotel_id': '353', 'review_id': ['48954', '37238', '37266', '37348']},
 {'hotel_id': '385', 'review_id': ['41529', '41510'

In [61]:


for i in range(len(return_data)):
    hotel_id = return_data[i]['hotel_id']
    review_id_list = return_data[i]['review_id']
    
    print(f'----- {hotel_info_df.loc[int(hotel_id),"hote_name"]} -----')

    for review_id in review_id_list[:number_review]:
        print(hotel_review_df.loc[int(review_id)]['review'])
    print('')


----- 씨크루즈호텔 속초 -----
바다뷰와 바다와 가까워서 산책하기 좋아요~~(바다쪽 방은 일출을 방에서 볼수 있어서 좋아요)
바다도 잘 보이고 깨끗하고 좋았습니다. 주차장 시설도 잘 되어있고 전체적으로 좋았어요.
위치가 좋음.  방이 깨끗하고 침대가 큼 해돋이 전망이 최고임
위치는 좋았으나 주변 상가환경이 다소 산만한 것이 아쉬움

----- 더화이트 호텔 -----
부킹닷컴 통해 할인 받았습니다 시설은 깨끗하고 좋습니다  천장이 높아서 넓게 느껴져 정말 좋았습니다
다락처럼 복층으로 되어있어 애들이 좋아할것 같아요 ~ (밑에층 이나 옆방들은 소음에 괴롭겠지만 ㅠ) 테라스에서 바라보는 뷰와 공기가 참 좋아요~ 위치도 괜찮고 가성비면에서는 나쁘지 않아요
가성비가 좋고 위치가 외진만큼 호텔내 부대시설이 잘되어있음. 씨유 포함 음식점 다수. 전체적인 위생상태가 깨끗하고 호텔인 만큼 호텔다운 구성을 갖춤 (어메니티 투숙객 서비스 등) 층고가 높아 객실이 작아도 답답하지 않고 침구류도 좋았음. 산속에 있어서 한여름에도 매우 시원하고 6월 말임에도 모기등 벌레가 별로 없었음. 양떼목장 등 관광지와도 가까움
층고가 높아서 확ㅡ트인 느낌과 깔끔한 분위기는 첫인상을 매우 좋게 만들어줍니다^^ 복층구조인데 ㅣㅣ살 조카와 함께했지만 굳이 올라갈 일은 없었네요. 근데 그런 거 있잖아요? 다락방있는 것만으로도 어른들에겐 옛운치 아이들에겐 나만의 공간ㅡ처음엔 거기가 자신의 아지트라고 하더니 허리를 굽혀 다녀야하는 2층이 불편했는지 더이상 오르지 않음^^ ㅡ고객의 감성을 신경쓴 느낌이라 아주 만족해했습니다. 아~그리고 조용해서 아주 좋았습니다~방배정 잘못 받으면 돈 내고 스트레스 쌓아서 가는 불운ㅜㅜ 아주 조용하고 시원하고 깔끔하고 함께 한 할머니와 조카군 모두 다음에 또 와보자 했을 정도입니다. 가격대비 최고라 감히 적어봅니다.

----- 롯데 리조트 속초 -----
해변과 가까이 있어서 산책하기가 매우 좋았음. 호텔 주위에 관광지가 가깝게 위치해 있어서 쉽게 관광지를 둘러 보

In [259]:
import re
from konlpy.tag import *
import pandas as pd

In [548]:
# csv file path
df = pd.read_csv("../../dataset/edited_for_db/여수_hotel_reviews.csv", encoding="UTF-8")

In [549]:
#리뷰 합치기
positive_df = df[['region', 'hotel', 'date', 'review_positive']].copy()
negative_df = df[['region', 'hotel', 'date', 'review_negative']].copy()

positive_df.rename(columns={'review_positive' : "review", 'hotel' : 'hotel_name'}, inplace=True)
negative_df.rename(columns={'review_negative' : "review", 'hotel' : 'hotel_name'}, inplace=True)

df = pd.concat([positive_df, negative_df], axis=0)

df['region'] = '여수'

In [550]:
df.dropna(inplace=True)

In [551]:
len(df)

9188

In [552]:
df.duplicated(['review', 'hotel_name', 'date']).sum()

28

In [553]:
df.drop_duplicates(['review', 'hotel_name', 'date'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [554]:
len(df)

9160

In [555]:
df.head(3)

,region,hotel_name,date,review
0,여수,베니키아 호텔 여수,2020년 12월,호텔 청결하고 좋았어요. 침대 크고 잠자리도 편했습니다. 바닥난방까지 되어서 따뜻했...
1,여수,베니키아 호텔 여수,2022년 2월,깨끗하고 편안했습니다.
2,여수,베니키아 호텔 여수,2022년 2월,바다전망 넘 좋았어요~


In [556]:
reviews = df["review"]

hannanum = Hannanum()
index = []

for i in range(len(reviews)):
    # print(i)
    # 줄바꿈 없애기
    inline_review = re.sub('\n', '', reviews[i])
    before_review = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', inline_review)
    # 단순 리뷰 삭제 (... 없음 딱히 등등)
    if len(before_review) < 15:
        index.append(i)
    else:
        review = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inline_review)
        tokens = hannanum.morphs(review)
        # 형태소 갯수 제한 - 5개 이상 50개 미만
        if len(tokens) < 5:
            print(review)
            index.append(i)
        elif len(tokens) > 50:
            print(review)
            index.append(i)
        pass

호텔 청결하고 좋았어요  침대 크고 잠자리도 편했습니다  바닥난방까지 되어서 따뜻했고 방도 크고 샤워실도 넓고 깨끗했어요  조식부페도 맛있었구요 스태프들도 모두 친절하고 가족들도 매우 만족했습니다  주변에 식당들과 편의점들이 있어서 편리했습니다 
건물 지하주차장 입구가 좁아 인근 공영주차장 이용료 제공  구시가지 식당가 근처 모텔촌에 위치해 밤 분위기가 좀 어수선했지만  오션뷰가 제일 좋았음 넓은 화장실과 객실 리뉴얼  인근 식사 및 산책 코스 친절히 안내해 준 직원 서비스  따뜻한 바닥 히팅과 에어컨디셔너도 좋았음 
넓고 깨끗했습니다 다만 너무 건조해서 자고 일어나면 눈이 아팠어요;; 수건에 물 적셔서 주변에 놓고 주무시거나 컵에 물을 받아놓고 주무셔야 할 듯 합니다ㅠㅠ 직원분들은 친절하세요 ㅎㅎ 조식도 나름 먹을만 했네용  2박3일 있었는데 물은 첫날만 제공되네요ㅠㅠ 그래서 이틀째 되는 날에는 물을 구매해야했어요  개당 천원입니다 
1 기대 이상으로 객실 상태가 쾌적 깔끔해 첫인상이 좋았음  2 객실 크기가 일반보다 넓어서 갑갑함이 없이 여유있게 공간을 머물 수 있었음  3 어메니티가 아주 유용하게 구성되어 있고 샤워용품은 향도 좋아 바로 잘 사용할 수 있었음  sewing kit도 있었음 더 좋았을텐데 나무 큰 욕심이겠죠   4 요즘 대부분 샤워부스만 있어 호텔 선택시 많이 살펴보는 편인데 베니키아는 피곤한 여독을 풀어줄 욕조가 딸려 있어 정말 좋았음  5 조식도 기대 이상으로 만족스러웠음  음식 가짓수 보다는 조화로운 음식 구성이 좋았음  맛도 음식마다 훌륭했음  2박 동안 메뉴도 달라져서 더욱 좋았음 
침구가 깨끗하고 이불도 푹신해서 잠을 잘잤어요  생각보다 룸이 조금 커요 냉장고가 작긴하나 엄청 시원하고 조용하고 뒷쪽에 관공서 건물이 있어서 가격대비 좋은 식당들이 많고 10분정도 걸어가면 시내와 스타벅스가 있어서 좋았어요
주변 상권이 발달해 있으나 시끄럽지 않은 식당 위주의 상권이며  산책하기 좋은 거리와 가까운 거리에 여행하기 좋은 곳이 있어 좋습

In [557]:
new_df = df.drop(index=index)
#new_df = df

In [558]:
new_df.head(3)

,region,hotel_name,date,review
3,여수,베니키아 호텔 여수,2022년 2월,아침식사는 매우 만족했습니다
4,여수,베니키아 호텔 여수,2022년 1월,호텔이 작은 건물 하나지만 깨끗햇습니다
5,여수,베니키아 호텔 여수,2022년 1월,"친절하고, 객실정비도 잘되어 너무 좋았습니다. 오션뷰도 저렴히 이용할수있어 만족스러..."


In [559]:
hotel_info = pd.read_csv("../../dataset/final/final_hotel_info.csv", encoding="UTF-8")

In [560]:
region_hotel_info = hotel_info[hotel_info['region'] == '여수']

In [561]:
info = region_hotel_info[['hotel_name', 'hotel_id']]

In [566]:
test = new_df.merge(info, on='hotel_name')
#test = new_df.merge(info, on='hotel_name')

In [567]:
test.sort_values('region')

,region,hotel_name,date,review,hotel_id
0,여수,베니키아 호텔 여수,2022년 2월,아침식사는 매우 만족했습니다,496
3834,여수,호텔 마띠유 여수,2019년 5월,깨끗하고 넓고 특히 화장실이 좋았어요. 방도 아늑하고 다음에도 여수 온다면 다시 올...,499
3833,여수,호텔 마띠유 여수,2019년 6월,호텔이 깨끗하고 분위기도 좋았습니다.,499
3832,여수,호텔 마띠유 여수,2019년 6월,새로 리모델링해서 깔끔하고 객실이 많지 않아 조용해요. 중간중간 테라스랑 정원이 있...,499
3831,여수,호텔 마띠유 여수,2019년 6월,전반적으로 아주 만족합니다. 정원이 아주 잘 관리 되어서 힐링이 되는 공간이 많아요,499
...,...,...,...,...,...
1910,여수,여수 베네치아 호텔 & 리조트,2020년 9월,"위치도,청결도,조용함도 모두 좋았어요.",489
1909,여수,여수 베네치아 호텔 & 리조트,2020년 11월,배네치아 괜히 베네치아가 아닙니다 베네치아 현지에 있는 느낌이 듭니다 시설도 너무 ...,489
1908,여수,여수 베네치아 호텔 & 리조트,2020년 11월,"호텔의 위치가 오동도 관광을 비롯한 해양케이블카, 낭만포차, 이순신 장군광장등 주요...",489
1928,여수,여수 베네치아 호텔 & 리조트,2020년 1월,침대가 너무 편해서 푹잘잤어요,489


In [568]:
len(test)

5744

In [569]:
len(new_df)

5744

In [570]:
new_df.to_csv("../../dataset/edited_for_db/modified_여수_review.csv", mode="w", header=True)

In [583]:
test = pd.read_csv('../../dataset/edited_for_db/modified_강원도_review.csv')

In [584]:
t1 = test.groupby('hotel_name').count()

In [585]:
t1[t1['region'] <= 5]

,Unnamed: 0,region,date,review
hotel_name,,,,
강릉 바다보기 펜션,3,3,3,3
산애가 펜션,5,5,5,5


In [586]:
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
import os

In [600]:
hotel_info_df = pd.read_csv('../../dataset/final/final_hotel_info.csv', encoding='utf-8')
hotel_review_df = pd.read_csv('../../dataset/final/final_hotel_review.csv', encoding='utf-8')

In [591]:
hotel_review_df.rename(columns={'review' :'contents'}, inplace=True)

In [596]:
hotel_review_df.head(3)

,review_id,date,contents,hotel_id,label
0,1,2021-06,조식이 세미 뷔페식이었어요. \n여자친구는 건강식 전 펜케익을 먹었었는데 맛은 매우...,1,1
1,2,2022-02,쇼핑몰과 연결되어 있어 이동이 편했습니다.,1,1
2,3,2022-02,조식 맛잇엇어요 직원분들이 친절햇습니다,1,1


In [597]:
positive_hotel_review_df = hotel_review_df[hotel_review_df['label'] == 1]

In [598]:
reindex_hotel_info_df = hotel_info_df.set_index('hotel_id')

In [601]:
hotel_info_df

,region,hotel_id,hotel_name,hotel_link,hotel_img_link
0,서울,1,콘래드 서울,https://www.booking.com/hotel/kr/conrad-seoul....,https://t-cf.bstatic.com/xdata/images/hotel/sq...
1,부산,328,해운대 센텀 호텔,https://www.booking.com/hotel/kr/haeundae-cent...,https://t-cf.bstatic.com/xdata/images/hotel/sq...
2,부산,329,신라스테이 서부산,https://www.booking.com/hotel/kr/shillastay-se...,https://t-cf.bstatic.com/xdata/images/hotel/sq...
3,부산,330,켄트호텔 광안리 by 켄싱턴,https://www.booking.com/hotel/kr/kent-gwangall...,https://t-cf.bstatic.com/xdata/images/hotel/sq...
4,부산,331,베스트 루이스 해밀턴 호텔 해운대,https://www.booking.com/hotel/kr/best-louis-ha...,https://cf.bstatic.com/xdata/images/hotel/squa...
...,...,...,...,...,...
219,여수,547,포유호스텔,https://www.booking.com/hotel/kr/poyu-hoseutel...,https://cf.bstatic.com/xdata/images/hotel/squa...
220,여수,548,여수 해비치 호텔,https://www.booking.com/hotel/kr/haebici.ko.ht...,https://cf.bstatic.com/xdata/images/hotel/squa...
221,여수,549,여수 mj관광호텔,https://www.booking.com/hotel/kr/yeosu-mjgwang...,https://cf.bstatic.com/xdata/images/hotel/squa...
222,여수,550,Yeosu Inn Hostel,https://www.booking.com/hotel/kr/yeosu-inn-gue...,https://cf.bstatic.com/xdata/images/hotel/squa...


In [603]:
okt = Okt()

In [604]:
okt.pos('가성비 좋은')

[('가성', 'Noun'), ('비', 'Noun'), ('좋은', 'Adjective')]